**REPOSITORIO UNSAAC**

- Alex Jesus Flores Taco   (20210833)



In [1]:
# Cargando librerías o módulos pertinentes
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd

In [2]:
# Creando una función que permite extraer todas las urls de las páginas que contienen
# las publicaciones encontradas en erespectivo repositorio
def all_pages(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene el url base de las páginas que contienen las publicaciones
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url base de las páginas que contienen las publicaciones
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["https://repositorio.unsaac.edu.pe/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de urls de cada una de las páginas 
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

In [3]:
all_pages('https://repositorio.unsaac.edu.pe/browse?type=dateissued')

['https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=0&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=20&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=40&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=60&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=80&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=100&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=120&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=140&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?rpp=20&sort_by=2&type=dateissued&offset=160&etal=-1&order=ASC',
 'https://repositorio.unsaac.edu.pe/browse?

In [4]:
# Creando una función que permite extraer todas las urls de la tabla de datos de las
# publicaciones de una determina página
def links_scrapy(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag padre de las publicaciones que contiene su respectivo url
    publication = soup.find_all("h4", {"class":"artifact-title"})
    # Extraer los url de los tags hijos respectivos
    publication_soup = BeautifulSoup(str(publication), 'html.parser')
    a_tags = publication_soup.find_all('a') # find all anchor elements
    hrefs = ["https://repositorio.unsaac.edu.pe" + a_tag['href'] + "?show=full" for a_tag in a_tags]
    return hrefs

In [5]:
links_scrapy('https://repositorio.unsaac.edu.pe/browse?type=dateissued')

['https://repositorio.unsaac.edu.pe/handle/20.500.12918/5935?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/5427?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/5796?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/6487?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/7055?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/4106?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1119?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/926?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1055?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1026?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1053?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1070?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1080?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12

In [ ]:
### TENER CUIDADO CON AL EJECUCIOND E ESTE CODIGO SON 5769 TESIS Y MI LAPOTP NO AGUANTA

# Extracción de todas las urls de la tabla de datos de todas las
# publicaciones del repositorio
# NOTA: no se recomienda ejecutar esta celda, ya que toma demasiado tiempo obtener una salida
every_page = all_pages('https://repositorio.unsaac.edu.pe/browse?type=dateissued')
publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page]))

In [6]:
# EJEMPLO DE EJECUCIÓN
# En este ejemplo solo se considera las dos primeras páginas del repositorio
every_page = all_pages('https://repositorio.unsaac.edu.pe/browse?type=dateissued')
publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page[0:2]]))

In [7]:
publications_links

['https://repositorio.unsaac.edu.pe/handle/20.500.12918/5935?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/5427?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/5796?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/6487?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/7055?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/4106?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1119?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/926?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1055?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1026?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1053?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1070?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1080?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12

In [9]:
## CREEAMOS  LISTAS PARA LA VARIABLES QUE DESEAMOS EXTRAER.

headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}


años = []
instituciones = []
titulos = []
autores = []
contribuidores = []
grados = []
resumenes = []
n = 0
for i in range(len(publications_links)):
    respuesta = requests.get(publications_links[i], headers=headers)
    tabla_tesis = pd.read_html(respuesta.content, encoding = 'utf8')
    tabla_1 = tabla_tesis[0]


    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.date.issued':
            año=tabla_1.iloc[j,1]
            años.append(año)
            n = n + 1
    if n == 0:
        años.append(" ")

    else:
        n = 0



    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.publisher':
            institucion=tabla_1.iloc[j,1]
            instituciones.append(institucion)
            n = n + 1
    if n == 0:
        instituciones.append(" ")

    else:
        n = 0



    for j in range(len(tabla_1)):        
        if tabla_1.iloc[j,0] == 'dc.title':
            titulo=tabla_1.iloc[j,1]
            titulos.append(titulo)
            n = n + 1
    if n == 0:
            titulos.append(" ")

    else:
        n = 0



    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.author':
            autor=tabla_1.iloc[j,1]
            autores.append(autor)
            n = n + 1
            break
    if n == 0:
            autores.append(" ")

    else:
        n = 0 



    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.name':
            grado=tabla_1.iloc[j,1]
            grados.append(grado)
            n = n + 1
    if n == 0:
            grados.append(" ")

    else:
        n = 0



    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.advisor':
            contribuidor=tabla_1.iloc[j,1]
            contribuidores.append(contribuidor)
            n = n + 1
            break
    if n == 0:
            contribuidores.append(" ")
    else:
        n = 0



    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.description.abstract':
            resumen=tabla_1.iloc[j,1]
            resumenes.append(resumen)
            n = n + 1
    if n == 0:
            resumenes.append(" ")
    else:
        n = 0
        

In [10]:
#creamos un dataframe para poder guardarlos en un archivo csv

df = pd.DataFrame()

df['instituciones'] = instituciones
df['titulos'] = titulos
df['autores'] = autores
df['contribuidores'] = contribuidores
df['resumenes'] = resumenes
df['grados'] = grados
df['años'] = años

In [11]:
#GUARDAMOS LA INFROMACION EN UN ARCHIVO CSV

df.to_csv('UNSAAC.csv', index=False)
df

,instituciones,titulos,autores,contribuidores,resumenes,grados,años
0,Universidad Nacional de San Antonio Abad del C...,Acción laboral y formación profesional de los ...,"Huaman Naula, Nicanor","Borda Pilinco, Manrique",La investigación que se presenta a continuació...,Maestro en Administración,2004
1,Universidad Nacional de San Antonio Abad del C...,Estabilidad estructural de sistemas dinámicos ...,"Choque Huaman, Patricio","Velasquez Hacha, Ignacio",El estado de un sistema dinámico se describe p...,Maestro en Matemáticas,2008
2,Universidad Nacional de San Antonio Abad del C...,Efectos de un programa de entrenamiento vocáli...,"Taipe Mendoza, Marisol Marlene","Pacheco Sota, Vilma Aurora",El objetivo del presente trabajo de investigac...,Título de Segunda Especialidad Profesional en ...,2008
3,Universidad Nacional de San Antonio Abad del C...,Estudio pronóstico de demanda de energía y pot...,"Flores Zambrano, Italo Roberto","GonzAles Vargas, Mario",En el presente informe Técnico se desarrolla e...,Ingeniero Electricista,2008
4,Universidad Nacional de San Antonio Abad del C...,Optimización de los sistemas de protección elé...,"Herrera Velasco, Jose Paul","Loaiza Cuba, Wilbert Julio",La experiencia en la operación de los equipos ...,Ingeniero Electricista,2009
5,Universidad Nacional de San Antonio Abad del C...,Resistencia de cuatro híbridos fhia de banano ...,"Depaz Molina, Karen Eva","Céspedes Del Pozo, Wilton Henry",La investigación se realizó en el Centro Agrar...,Ingeniero Agrónomo Tropical,2010
6,Universidad Nacional de San Antonio Abad del C...,Efectos de las plantas espontáneas o K'ipas co...,"Huamán Pante, Yanet Mercedes","Catalán Bazán, Wilfredo",La pérdida de los sistemas de cultivo sectoria...,Ingeniero Agrónomo,2010
7,Universidad Nacional de San Antonio Abad del C...,Las tasas del impuesto a la renta por las rent...,"Quispe Sencia, Frine Epifania","Abrill Góngora, Edmundo",El Sistema Tributario en el Perú está conforma...,Contador Público,2011
8,Universidad Nacional de San Antonio Abad del C...,Actividad antibacteriana In Vitro de los extra...,"Castro Sumaria, Verónica","Del Carpio Jiménez, Carla",El presente trabajo tiene por finalidad demost...,Químico Farmacéutico,2011
9,Universidad Nacional de San Antonio Abad del C...,Plan de gestión de riego en la subcuenca de Pa...,"Quispe Quispe, Jesús Macario","Huamán Miranda, Guido Vicente",El presente trabajo de investigación denominad...,Ingeniero agrónomo,2011
